<a href="https://colab.research.google.com/github/kimp10/DATASCIENCE_PROJECTS/blob/main/lacuna_improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kimp1995_lacuna_solar_survey_challenge_path = kagglehub.dataset_download('kimp1995/lacuna-solar-survey-challenge')

print('Data source import complete.')


Data source import complete.


In [28]:
pip install ultralytics --upgrade


In [29]:
from ultralytics import YOLO
import os
import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import glob
import torch
from tqdm import tqdm
import gc


In [30]:
# Load the training dataset
train_csv_path = os.path.join(kimp1995_lacuna_solar_survey_challenge_path, "Train.csv")
test_csv_path = os.path.join(kimp1995_lacuna_solar_survey_challenge_path, "Test.csv")


In [31]:
import os

# Define the images directory
images_path = os.path.join(kimp1995_lacuna_solar_survey_challenge_path, "images")

# List some image files
image_files = os.listdir(images_path)[:10]  # Show first 10 images
print("Sample Images:", image_files)


Sample Images: ['ID6j28Vj6pux7AM.jpg', 'IDDnX91KjDPn57N.jpg', 'IDYOAM3GL0qBM.jpg', 'IDgTAXk.jpg', 'IDsq0cQY.jpg', 'IDEU9ZFLr0.jpg', 'IDic5ODIq4o.jpg', 'IDKVu0o7UiNsnD0Kk.jpg', 'IDgQQig.jpg', 'ID72JDsOX11U.jpg']


In [32]:
# Define paths
ROOT_DIR = "/content/sample_data"
IMG_DIR = os.path.join(images_path, "images")

# Create required folders
for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(ROOT_DIR, folder), exist_ok=True)

# Split dataset (80% train, 20% val)
df = pd.read_csv(train_csv_path)
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

# Create symbolic links for images
for subset, data in [("train", train_df), ("val", val_df)]:
    for _, row in tqdm(data.iterrows(), total=len(data)):
        img_path = os.path.join(IMG_DIR, row["ID"] + ".jpg")
        symlink_path = os.path.join(ROOT_DIR, "images", subset, row["ID"] + ".jpg")

        if os.path.exists(img_path):
            # Remove existing symlink if it exists
            if os.path.exists(symlink_path) or os.path.islink(symlink_path):
                os.unlink(symlink_path)  # Remove old symlink
            os.symlink(img_path, symlink_path)  # Create new symlink


100%|██████████| 1358/1358 [00:00<00:00, 20388.32it/s]


In [33]:
# Define class mapping
CLASS_MAPPING = {"pan_nbr": 0, "boil_nbr": 1}

import numpy as np

import ast  # More secure than eval()

def convert_polygon_to_yolo(polygon, img_width, img_height):
    """Convert polygon coordinates to YOLO format (x_center, y_center, width, height)."""
    if pd.isna(polygon) or polygon == "nan":  # Check for NaN values
        return None, None, None, None  # Return empty values if polygon is missing

    # Replace 'nan' with 'None' to prevent errors
    safe_polygon = polygon.replace("nan", "None") if isinstance(polygon, str) else polygon

    try:
        polygon = ast.literal_eval(safe_polygon)  # Safely convert string to list
    except (SyntaxError, ValueError):
        return None, None, None, None  # Handle cases where conversion fails

    # **Filter out None values**
    polygon = [p for p in polygon if p is not None and len(p) == 2 and all(isinstance(coord, (int, float)) for coord in p)]

    if not polygon:  # If empty after filtering, return None
        return None, None, None, None

    x_min = min(p[0] for p in polygon)
    x_max = max(p[0] for p in polygon)
    y_min = min(p[1] for p in polygon)
    y_max = max(p[1] for p in polygon)

    # Convert to YOLO format
    x_c = (x_min + x_max) / 2 / img_width
    y_c = (y_min + y_max) / 2 / img_height
    w_c = (x_max - x_min) / img_width
    h_c = (y_max - y_min) / img_height

    return x_c, y_c, w_c, h_c

# Generate YOLO labels
for subset, data in [("train", train_df), ("val", val_df)]:
    for _, row in tqdm(data.iterrows(), total=len(data)):
        img_path = os.path.join(IMG_DIR, row["ID"] + ".jpg")
        if not os.path.exists(img_path):
            continue

        h, w, _ = cv2.imread(img_path).shape  # Get image dimensions

        yolo_annotations = []
        for obj, class_id in CLASS_MAPPING.items():
            if row[obj] > 0:
                x_c, y_c, w_c, h_c = convert_polygon_to_yolo(row["polygon"], w, h)
                yolo_annotations.append(f"{class_id} {x_c} {y_c} {w_c} {h_c}")

        # Save annotation file
        label_path = os.path.join(ROOT_DIR, "labels", subset, row["ID"] + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(yolo_annotations))


100%|██████████| 1358/1358 [00:00<00:00, 21275.53it/s]


In [34]:
import torch

# Check if GPU is available
print(f"CUDA Available: {torch.cuda.is_available()}")

# Check current GPU info
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Running on CPU.")

# Check TPU status
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    print(f"TPU Available: {xm.xla_device()}")
except ImportError:
    print("TPU not available.")


Mounted at /content/drive


In [35]:
data_yaml = {
    "train": f"{ROOT_DIR}/images/train",
    "val": f"{ROOT_DIR}/images/val",
    "nc": 2,
    "names": ["solar_panel", "solar_boiler"]
}
with open(f"{ROOT_DIR}/data.yaml", "w") as f:
    yaml.dump(data_yaml, f)


In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
from ultralytics import YOLO

# Select TPU device
device = xm.xla_device()

# Load YOLOv11 model (Ensure this exists or use best checkpoint)
model = YOLO("yolo11n.pt")

# Train the model with improved settings
model.train(
    data="/content/sample_data/data.yaml",  # Path to dataset YAML
    epochs=100,  # Train longer
    imgsz=1024,  # Higher resolution for better small object detection
    batch=32,  # Increase batch size if GPU allows
    optimizer="AdamW",  # Better optimizer
    lr0=0.002,  # Adjusted learning rate
    momentum=0.937,
    weight_decay=0.0005,
    patience=20,  # Allow early stopping
    device="cuda" if torch.cuda.is_available() else "cpu",

    # 💡 Advanced Augmentations
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,  # Rotations
    translate=0.2,  # Translation
    scale=0.8,  # Zoom-in/out
    shear=10.0,  # Shear augmentation
    perspective=0.001,
    fliplr=0.5,  # Flip images horizontally
    flipud=0.1,  # Flip vertically (low probability)
    mosaic=1.0,  # YOLO mosaic augmentation
    mixup=0.1,  # Mixup augmentation
    auto_augment="randaugment",  # Auto Augmentation
    copy_paste=0.2,  # Copy-Paste augmentation

    # 📌 Fine-Tuning
    resume=False,  # Continue from last checkpoint
    overlap_mask=True,  # Improve segmentation
    mask_ratio=4,
    label_smoothing=0.05,  # Reduce false positives

    # 🔍 Adjust Confidence Threshold for better detection
    conf=0.2,  # Lower confidence threshold for detecting small objects
    iou=0.5,  # Adjust IoU threshold
)


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/sample_data/data.yaml, epochs=100, time=None, patience=20, batch=32, imgsz=1024, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

train: Scanning /content/sample_data/labels/train.cache... 2901 images, 0 backgrounds, 10 corrupt: 100%|██████████| 2901/2901 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/sample_data/images/train/ID5vE00zAieZ.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0202]
train: WARNING ⚠️ /content/sample_data/images/train/ID6xfy4L1YW.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      3.728        6.21]
train: WARNING ⚠️ /content/sample_data/images/train/IDFBDaBjXctMXwGX.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1862      1.2285]
train: WARNING ⚠️ /content/sample_data/images/train/IDJjksJEUnO.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      3.418      5.7533]
train: WARNING ⚠️ /content/sample_data/images/train/IDL6q17JM2MED3lF.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     2.0062      3.3026]
train: WARNING ⚠️ /content/sample_data/images/train/IDNb2NQUUl5wIdi.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0


val: Scanning /content/sample_data/labels/val.cache... 1044 images, 0 backgrounds, 3 corrupt: 100%|██████████| 1044/1044 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/sample_data/images/val/IDQT3ndJeVMOYKvm.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0815]
val: WARNING ⚠️ /content/sample_data/images/val/IDS4V13CBJiOe.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.4053      5.7436]
val: WARNING ⚠️ /content/sample_data/images/val/IDdtoqJ2zSLQ5mv0q.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.066]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: AdamW(lr=0.002, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.924        4.8      1.723         69       1024:  10%|▉         | 9/91 [05:15<46:32, 34.05s/it]

In [ ]:
metrics = model.val()
print(metrics)


In [ ]:
 #Load the Test.csv file
test_csv_path = os.path.join(kimp1995_lacuna_solar_survey_challenge_path, "Test.csv")
test_df = pd.read_csv(test_csv_path)

In [ ]:
# Get the list of image IDs from Test.csv
test_image_ids = test_df["ID"].tolist()

# Set batch size to process images in smaller chunks
BATCH_SIZE = 50  # Adjust if needed

# Load the trained model
model = YOLO("/content/sample_data/runs/detect/train/weights/best.pt")  # Change path if needed

# Initialize list for predictions
predictions = []

# Process test images in small batches to reduce memory usage
for i in range(0, len(test_image_ids), BATCH_SIZE):
    batch = test_image_ids[i:i + BATCH_SIZE]  # Get batch of image IDs
    batch_paths = [os.path.join(IMG_DIR, f"{img_id}.jpg") for img_id in batch if os.path.exists(os.path.join(image_dir, f"{img_id}.jpg"))]

    # Run inference on batch
    results = model.predict(source=batch_paths, device="cuda" if torch.cuda.is_available() else "cpu", stream=True)

    # Process results
    for r in results:
        image_name = os.path.basename(r.path).replace(".jpg", "")  # Extract image ID

        # If detections exist
        if len(r.boxes) > 0:
            for box in r.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())

                # Append result
                predictions.append([image_name, cls, conf])
        else:
            # No detections
            predictions.append([image_name, 0, 0])  # Default to zero detections

    # **Free up memory**
    del results
    torch.cuda.empty_cache()
    gc.collect()

# Convert to DataFrame
df_predictions = pd.DataFrame(predictions, columns=["ID", "Class", "Confidence"])

In [ ]:
df_predictions = pd.DataFrame(predictions, columns=["ID", "Class", "Confidence"])

In [ ]:
df_predictions['Class'].value_counts()

In [ ]:
# Define class mapping
CLASS_MAPPING = {0: "pan", 1: "boil"}  # 0 -> Solar Panel, 1 -> Solar Boiler

# Convert Class ID to column names
df_predictions["Class"] = df_predictions["Class"].map(CLASS_MAPPING)

# Sum detections per image per class
df_submission = df_predictions.pivot_table(index="ID", columns="Class", values="Confidence", aggfunc="sum").reset_index()

# Ensure both 'boil' and 'pan' columns exist, filling missing values with 0.0 (as float)
for col in ["boil", "pan"]:
    if col not in df_submission.columns:
        df_submission[col] = 0.0  # Ensure missing class entries have a value of 0.0

# Convert confidence scores to float (not integer)
df_submission[["boil", "pan"]] = df_submission[["boil", "pan"]].fillna(0.0).astype(float)

# Transform the DataFrame to match submission format
submission_rows = []

for _, row in df_submission.iterrows():
    submission_rows.append({"ID": f"{row['ID']}_boil", "Target": row["boil"]})
    submission_rows.append({"ID": f"{row['ID']}_pan", "Target": row["pan"]})

# Convert to DataFrame
df_final_submission = pd.DataFrame(submission_rows)

In [ ]:
df_final_submission.head()

In [ ]:
df_final_submission

In [ ]:
# Save submission file
submission_path = "/content/sample_data/submissionyolo11.csv"
df_final_submission.to_csv(submission_path, index=False)

print(f"✅ Submission file saved at: {submission_path}")